![image.png](attachment:image.png)

##### **Good luck with taking your exam. Keep working and make your dreams all come true. Seeing the results of all of your hard work will make this struggle worth it. We’re all thinking of you.** 
<b><font color='blue'>AI-PRO Spark Team ITI</font></b>

# NLP Using PySpark

## Objective:
- The objective from this project is to create a <b>Spam filter using NaiveBayes classifier</b>.
- It is required to obtain <b>f1_scored > 0.9</b>.
- We'll use a dataset from UCI Repository. SMS Spam Detection: https://archive.ics.uci.edu/ml/datasets/SMS+Spam+Collection
- Data is also provided for you in the assignment (you do not have to download it).

## To perform this task follow the following guiding steps:

### Create a spark session and import the required libraries

In [2]:
!pip install pyspark
!pip install findspark
!pyspark --version

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 42 kB/s 
     |████████████████████████████████| 199 kB 45.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=de66eaa197020f113b9fdcc9c488b3cc8731c65e6a961d64410fe8b8d336718c
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /___/ .__/\_,_/_/ /_/\_\   version 3.3.0
      /_/
                        
Using Scala version 2.12.15, OpenJDK 64-Bit Server VM, 11.0.15
Branch HEAD
Compiled by user ubuntu on 2022-06-09T19:58:58Z
Revision f74867bddfbcdd4d08076db36851e88b15e66556
Url https://github.com/apache/spark
Type --h

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [2]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SpamClassifier').getOrCreate()

### Read the readme file to learn more about the data

### Read the data into a DataFrame

In [4]:
# Load data and rename column
df = spark.read.option("header", "false") \
    .option("delimiter", "\t") \
    .option("inferSchema", "true") \
    .csv("/content/SMSSpamCollection")

df.show(10)

+----+--------------------+
| _c0|                 _c1|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
+----+--------------------+
only showing top 10 rows



### Print the schema

In [5]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



### Rename the first column to 'class' and second column to 'text'

In [6]:
df = df.withColumnRenamed("_c0", "class") \
       .withColumnRenamed("_c1", "text")

df.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)



### Show the first 10 rows from the dataframe
- Show once with truncate=True and once with truncate=False

In [7]:
df.show(10)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
+-----+--------------------+
only showing top 10 rows



+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                            |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...                                                 |
|ham  |Ok lar... Joking wif u oni...                                                                                                                                   |
|spam |Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075o

## Clean and Prepare the Data

### Create a new feature column contains the length of the text column

In [8]:
df = df.withColumn("text_len",length("text"))

### Show the new dataframe

In [9]:
df.show()

+-----+--------------------+--------+
|class|                text|text_len|
+-----+--------------------+--------+
|  ham|Go until jurong p...|     111|
|  ham|Ok lar... Joking ...|      29|
| spam|Free entry in 2 a...|     155|
|  ham|U dun say so earl...|      49|
|  ham|Nah I don't think...|      61|
| spam|FreeMsg Hey there...|     147|
|  ham|Even my brother i...|      77|
|  ham|As per your reque...|     160|
| spam|WINNER!! As a val...|     157|
| spam|Had your mobile 1...|     154|
|  ham|I'm gonna be home...|     109|
| spam|SIX chances to wi...|     136|
| spam|URGENT! You have ...|     155|
|  ham|I've been searchi...|     196|
|  ham|I HAVE A DATE ON ...|      35|
| spam|XXXMobileMovieClu...|     149|
|  ham|Oh k...i'm watchi...|      26|
|  ham|Eh u remember how...|      81|
|  ham|Fine if thats th...|      56|
| spam|England v Macedon...|     155|
+-----+--------------------+--------+
only showing top 20 rows



### Get the average text length for each class (give alias name to the average length column)

In [10]:
df.groupby('class').agg(avg("text_len").alias("Avg. Length")).show()

+-----+-----------------+
|class|      Avg. Length|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

### In this part you transform you raw text in to tf_idf model :
- For more information about TF-IDF check the following link: <b>(Not needed for the test)</b>
https://en.wikipedia.org/wiki/Tf%E2%80%93idf

### Perform the following steps to obtain TF-IDF:
1. Import the required transformers/estimators for the subsequent steps.
2. Create a <b>Tokenizer</b> from the text column.
3. Create a <b>StopWordsRemover</b> to remove the <b>stop words</b> from the column obtained from the <b>Tokenizer</b>.
4. Create a <b>CountVectorizer</b> after removing the <b>stop words</b>.
5. Create the <b>TF-IDF</b> from the <b>CountVectorizer</b>.

In [11]:
from pyspark.ml.feature import Tokenizer,StopWordsRemover, CountVectorizer,IDF,StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import NaiveBayes

In [12]:
df_clean = df.withColumn('text', (lower(regexp_replace('text', "[^\w\s']|http\S+|www\S+", "")).alias('text')))


In [13]:
from os import truncate
df_clean.show(truncate=False)

+-----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|class|text                                                                                                                                                                                             |text_len|
+-----+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|ham  |go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat                                                                                           |111     |
|ham  |ok lar joking wif u oni                                                                                                                              

In [14]:
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")

In [15]:
stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')

In [16]:
count_vec = CountVectorizer(inputCol='stop_tokens',outputCol='c_vec')

In [17]:
idf = IDF(inputCol="c_vec", outputCol="tf_idf")

In [18]:
# ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label')
class_index = StringIndexer(inputCol='class',outputCol='label')

In [19]:
vec_assemb = VectorAssembler(inputCols=['tf_idf','text_len'],outputCol='features')

In [65]:
# Past Check Output 

+-----+--------------------+--------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|class|                text|text_len|label|          token_text|         stop_tokens|               c_vec|              tf_idf|            features|
+-----+--------------------+--------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+
|  ham|go until jurong p...|     111|  0.0|[go, until, juron...|[go, jurong, poin...|(9463,[7,13,35,54...|(9463,[7,13,35,54...|(9464,[7,13,35,54...|
|  ham|ok lar joking wif...|      29|  0.0|[ok, lar, joking,...|[ok, lar, joking,...|(9463,[1,10,216,3...|(9463,[1,10,216,3...|(9464,[1,10,216,3...|
| spam|free entry in 2 a...|     155|  1.0|[free, entry, in,...|[free, entry, 2, ...|(9463,[3,9,21,25,...|(9463,[3,9,21,25,...|(9464,[3,9,21,25,...|
|  ham|u dun say so earl...|      49|  0.0|[u, dun, say, so,...|[u, dun, say, ear...|(9463,[1,68,72,83...|

- Convert the <b>class column</b> to index using <b>StringIndexer</b>
- Create feature column from the <b>TF-IDF</b> and <b>lenght</b> columns.

## The Model
- Create a <b>NaiveBayes</b> classifier with the default parameters.

In [20]:
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes(featuresCol='features',labelCol='label',predictionCol='prediction')

## Pipeline
### Create a pipeline model contains all the steps starting from the Tokenizer to the NaiveBays classifier.

In [28]:
(training,testing) = df_clean.randomSplit([0.7,0.3],seed=42)


In [29]:
training.show()

+-----+--------------------+--------+
|class|                text|text_len|
+-----+--------------------+--------+
|  ham|                    |       3|
|  ham|                    |       7|
|  ham|    all write or wat|      20|
|  ham| am on a train ba...|      56|
|  ham|        am on my way|      14|
|  ham| and dont worry w...|      53|
|  ham| bot notes oredi ...|      43|
|  ham|   but your not here|      24|
|  ham|    called dad oredi|      21|
|  ham| comin to fetch u...|      28|
|  ham| dun need to pick...|      25|
|  ham| dun wan to watch...|      35|
|  ham| give me some tim...|      34|
|  ham| gonna let me kno...|      95|
|  ham| im  on the snowb...|     146|
|  ham| log off 4 wat it...|      29|
|  ham| ltdecimalgt m bu...|     132|
|  ham| neva tell me how...|      61|
|  ham|   oh well c u later|      22|
|  ham| ow u deyi paid 6...|      55|
+-----+--------------------+--------+
only showing top 20 rows



In [30]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[class_index,tokenizer,stopremove,count_vec,idf,vec_assemb,nb])
train_pred = pipeline.fit(training).transform(training)


In [31]:
train_pred.show()

+-----+--------------------+--------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|class|                text|text_len|label|          token_text|         stop_tokens|               c_vec|              tf_idf|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  ham|                    |       3|  0.0|                  []|                  []|        (7951,[],[])|        (7951,[],[])| (7952,[7951],[3.0])|[-1.6296793588199...|[0.88877876138000...|       0.0|
|  ham|                    |       7|  0.0|                  []|                  []|        (7951,[],[])|        (7951,[],[])| (7952,[7951],[7.0])|[-3.6098771483600...|[0.91434881610405...|  

### Split your data to trian and test data with ratios 0.7 and 0.3 respectively.

### Fit your Pipeline model to the training data

### Perform predictions on tests dataframe

In [32]:
test_pred = pipeline.fit(testing).transform(testing)

In [33]:
test_pred.show()

+-----+--------------------+--------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|class|                text|text_len|label|          token_text|         stop_tokens|               c_vec|              tf_idf|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  ham|   and  picking t...|     169|  0.0|[, , , and, , pic...|[, , , , picking,...|(4350,[0,1,3,8,21...|(4350,[0,1,3,8,21...|(4351,[0,1,3,8,21...|[-945.34205529368...|[1.0,3.6463121768...|       0.0|
|  ham| and  picking the...|      41|  0.0|[, and, , picking...|[, , picking, var...|(4350,[0,411,994,...|(4350,[0,411,994,...|(4351,[0,411,994,...|[-199.06266137595...|[0.99766306801544...|  

### Print the schema of the prediction dataframe

In [34]:
train_pred.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- text_len: integer (nullable = true)
 |-- label: double (nullable = false)
 |-- token_text: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stop_tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- c_vec: vector (nullable = true)
 |-- tf_idf: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)
 |-- label: double (nullable = false)
 |-- token_text: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stop_tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- c_vec: vector (nullable = true)
 |-- tf_idf: vector (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



## Model Evaluation
- Use <b>MulticlassClassificationEvaluator</b> to calculate the <b>f1_score</b>.

In [35]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_eval = MulticlassClassificationEvaluator(metricName='f1')
acc = acc_eval.evaluate(test_pred)
print("Accuracy of model at predicting spam was: {}".format(acc))


Accuracy of model at predicting spam was: 0.9956358767266815


f1_score is: 0.9664707489549014


# GOOD LUCK
<b><font color='GREEN'>AI-PRO Spark Team ITI</font></b>

![image-3.png](attachment:image-3.png)